## Constants

In [19]:
CHANNELS = ['red', 'green', 'blue', 'yellow']
TRAIN_CSV = 'D:/HPA_comp/single_cells/train_folds_big.csv'
IMG_DIR = 'D:/HPA_comp/single_cells'
MEAN_CHANNEL_VALUES = [0.485, 0.456, 0.406]
CHANNEL_STD_DEV = [0.229, 0.224, 0.225]

In [2]:
from sklearn.metrics import auc, average_precision_score
from statistics import mean

def skl_mAP(y_preds, y_targets):
    'Return sklearn mean average precision across valid labels'
    ap_rec = []
    # Calc avg precision score one label at a time
    for lab in range(y_targets.shape[1]):
        y_pred = y_preds[:, lab]
        y_target = y_targets[:, lab]
        # If no targets present, skip label to avoid /0 runtime warning
        if y_target.sum() == 0:
            continue
        ap = average_precision_score(y_target, y_pred)
        ap_rec.append(ap)
    if len(ap_rec) == 0:
        return np.nan
    mean_AP = mean(ap_rec)
    return mean_AP

## Dataset class

In [3]:
import torch
import pandas as pd
import numpy as np
import imageio
from skimage.transform import resize
import os


class CellDataset(object):
    '''Dataset class to fetch HPA cell-level images
    and corresponding weak labels
    '''
    def __init__(self, images, targets, img_root, augmentations=None):
        self.images = images
        self.targets = targets
        self.img_root = img_root
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_id = self.images[idx] 
        img_channels = self._fetch_channels(img_id)
        img = self._channels_2_array(img_channels)
        img = resize(img, (260, 260))  # Always resize cell images for collate function
        # If augmentation pipeline provided, apply augmentations
        if self.augmentations:
            img = self.augmentations(image=img)['image']
        # Adjust to channel first indexing for pytorch (speed reasons)
        features = np.transpose(img, (2, 0, 1)).astype(np.float32)
        target = self.targets[idx]  # Grab target vector
        
        return {'image': torch.tensor(features),
                'target': torch.tensor(target)
                }
    
    def _fetch_channels(self, img_id: str, channel_names=CHANNELS):
        'Return absolute path of segmentation channels of a given image id'
        base = os.path.join(self.img_root, img_id)
        return [base + '_' + i  + '.png' for i in channel_names]
                                         
    def _channels_2_array(self, img_channels):
        'Return 3D array of pixel values of input image channels'
        r = imageio.imread(img_channels[0])
        g = imageio.imread(img_channels[1])
        b = imageio.imread(img_channels[2])
        pixel_arr = np.dstack((r, g, b))
        return pixel_arr

## Model class

In [4]:
from torch import nn
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import tez

#from custom_metrics import skl_mAP


class EfficientNetB2(tez.Model):
    '''Model class to facilitate transfer learning 
    from a resnet-18 model
    '''
    NUM_CLASSES = 19
    DROPOUT_RATE = 0.1
    IMG_DIR = 'D:/HPA_comp/single_cells'
    
    def __init__(self, train_df, valid_df, batch_size, train_aug=None, valid_aug=None, pretrained=True):
        super().__init__()
        # Initialise pretrained net and sub-in final layers for cell classification
        self.effnet = self.load_effnet(pretrained)
        self.effnet._fc = nn.Linear(1408, self.NUM_CLASSES)
        self.out = nn.Sigmoid()
        self.step_scheduler_after = "epoch"
        self.loss_fn = nn.BCELoss()
        
        # Below should probably be in tez.Model super class but is a quick hack around
        self.train_loader = self.gen_dataloader(train_df, batch_size, shuffle=True, aug=train_aug)
        self.valid_loader = self.gen_dataloader(valid_df, batch_size, shuffle=False, aug=valid_aug)
        
    def forward(self, image, target=None):
        # Forward prob on effnet model w/ sigmoid activations final layer
        x = self.effnet(image)
        output = self.out(x)
        
        if target is not None:
            loss = self.loss_fn(output, target.to(torch.float32))  # why to float32???
            metrics = self.monitor_metrics(output, target)
            return output, loss, metrics
        return output, None, None
    
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        targets = targets.cpu().detach().numpy()
        outputs = outputs.cpu().detach().numpy()
        # Calculate batch metrics
        mean_AP = skl_mAP(outputs, targets)
        return {'mAP': mean_AP}
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch
    
    def gen_dataloader(self, df, bs, shuffle, aug=None):
        'Return pytorch dataloader generated from cell image dataframe'
        # Extract images and targets as numpy arrays from dataframe tranche
        def extract_as_array(str_):
            list_ = str_.strip('][').split(', ')
            return np.array([int(i) for i in list_])
        images = df['cell_id'].values
        targets = df['Label'].apply(extract_as_array).values
        # Init custom dataset class and pass to pytorch
        dataset = CellDataset(images, targets, self.IMG_DIR, aug)
        return DataLoader(dataset, batch_size=bs, shuffle=shuffle)
    
    def load_effnet(self, pretrained):
        if pretrained == True:
            effnet = EfficientNet.from_pretrained("efficientnet-b2")
        else:
            effnet = EfficientNet.from_name("efficientnet-b2")
        return effnet

## if __name__ == '__main__'

In [5]:
import albumentations as A

# Image augmentation stack 
train_aug = A.Compose([
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Normalize(
        mean=MEAN_CHANNEL_VALUES,
        std=CHANNEL_STD_DEV,
        max_pixel_value=1.0,
        p=1.0
    )
])

valid_aug = A.Compose([
    A.Normalize(
        mean=MEAN_CHANNEL_VALUES,
        std=CHANNEL_STD_DEV,
        max_pixel_value=1.0,
        p=1.0
    )
])

In [6]:
# Select training folds from csv
dfx = pd.read_csv(TRAIN_CSV, index_col=0)   #.iloc[:100, :]
FOLD = 0

df_train = dfx[dfx['fold'] != FOLD].reset_index(drop=True)
df_valid = dfx[dfx['fold'] == FOLD].reset_index(drop=True)

# Init model
model = EfficientNetB2(
     df_train,
     df_valid, 
     batch_size=16, 
     train_aug=train_aug, 
     valid_aug=valid_aug, 
     pretrained=True
)

# Load model from checkpoint
#model_path = '../models/model_checkpoint_effnet.bin'
#model.load(model_path)

# Early stopping
from tez.callbacks import EarlyStopping
es = EarlyStopping(
    monitor='valid_loss',
    model_path='../models/effnetb2_checkpoint.bin',
    patience=3,
    mode='min',
)

# Model training
model.fit(
    train_dataset=None,  # dataset inits are overriden in the model class above
    valid_dataset=None,  # otherwise tez breaks for me when it tries to do this itself
    train_bs=16,
    device='cuda',
    callbacks=[es],
    epochs=5
)

# Save model (with optimizer and scheduler for future usage)
model.save('../models/trained_effnetb2.bin')

Loaded pretrained weights for efficientnet-b2


  0%|                                                                                                                    | 0/3 [00:02<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 6.00 GiB total capacity; 4.08 GiB already allocated; 11.73 MiB free; 4.15 GiB reserved in total by PyTorch)